In [1]:
from Bio.Blast import NCBIXML
from Bio.Blast import NCBIWWW
from Bio import SeqIO

In [2]:
protein = SeqIO.read( "NP_041982.1.fasta", "fasta")

In [3]:
result_handle = NCBIWWW.qblast("blastp", "nr", protein.format("fasta"))

In [4]:
save_file = open("blast.xml", "w")
save_file.write(result_handle.read())
save_file.close()
result_handle.close()
result_handle = open("blast.xml")
blast_record = NCBIXML.read(result_handle)

In [5]:
print ("number hits: ", len(blast_record.alignments))
first_alignment = blast_record.alignments[0]

print ("FIRST ALIGNMENT:")
print ("Accession: " + first_alignment.accession)
print ("Hit id: " + first_alignment.hit_id)
print ("Definition: " + first_alignment.hit_def)
print ("Alignment length: ", first_alignment.length)
print ("Number of HSPs: ", len(first_alignment.hsps))

number hits:  50
FIRST ALIGNMENT:
Accession: QZB83852
Hit id: gb|QZB83852.1|
Definition: DNA polymerase [Escherichia phage T7]
Alignment length:  738
Number of HSPs:  1


In [22]:
blast_record = NCBIXML.read(open("blast.xml"))

BEST_HITS = []

for alignment in blast_record.alignments:
    if "Escherichia phage T7" in alignment.hit_def:
        continue

    hsp = alignment.hsps[0]

    if hsp.expect <= 1e-20:
        BEST_HITS.append((
            alignment.accession,
            alignment.hit_def,
            hsp.expect,
            hsp.sbjct
        ))

BEST_HITS = sorted(BEST_HITS, key=lambda x: x[2])[:10]

for hit in BEST_HITS:
    print("Accession:", hit[0])
    print("Definition:", hit[1])
    print("E-value:", hit[2])
    print()


Accession: YP_009799297
Definition: DNA polymerase I [Yersinia phage YpsP-G] >gb|AFK13511.1| T7-like DNA polymerase [Yersinia phage YpsP-G]
E-value: 0.0

Accession: UXY92503
Definition: DNA polymerase [Escherichia phage 04922B]
E-value: 0.0

Accession: YP_009813785
Definition: DNA polymerase I [Escherichia phage N30] >ref|YP_009813829.1| DNA polymerase I [Escherichia phage C5] >gb|AYD80145.1| DNA polymerase [Escherichia phage N30] >gb|AYD80189.1| DNA polymerase [Escherichia phage C5] >gb|YBW06339.1| hypothetical protein UHEC_00066 [Escherichia phage UHEC]
E-value: 0.0

Accession: XPK42555
Definition: DNA polymerase I [Escherichia phage MeretOppenheim_Bas102]
E-value: 0.0

Accession: YP_002003958
Definition: DNA polymerase I [Escherichia phage 13a] >gb|ACF15908.1| gp5 [Escherichia phage 13a]
E-value: 0.0

Accession: WP_113998065
Definition: DNA polymerase [Escherichia coli] >gb|RCF24106.1| DNA polymerase [Escherichia coli]
E-value: 0.0

Accession: YP_009798021
Definition: DNA polymerase

In [18]:
with open("best_hits.fasta", "w") as f:
    for acc, desc, evalue, seq in BEST_HITS:
        header = f"{acc}|evalue={evalue}"
        f.write(f">{header}\n{seq}\n\n")


In [21]:
records = list(SeqIO.parse("best_hits.fasta", "fasta"))
query = SeqIO.read("NP_041982.1.fasta", "fasta")

query.id = "NP_041982_1"
query.name = "NP_041982_1"
query.description = ""

records.insert(0, query)

SeqIO.write(records, "best_hits_with_query.fasta", "fasta")



11